IMPORT LIBRARIES

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


PART 1

Creating Toy Data & Model

In [ ]:

torch.manual_seed(0)

model = nn.Linear(in_features=2, out_features=1)

X = torch.tensor([[1.0, 2.0],
                  [2.0, 3.0],
                  [3.0, 4.0],
                  [4.0, 5.0]])

y = torch.tensor([[1.0],
                  [1.0],
                  [0.0],
                  [0.0]])


Forward Pass & Loss

In [ ]:

outputs = model(X)

criterion = nn.MSELoss()
loss = criterion(outputs, y)

print("Model Outputs:\n", outputs)
print("Loss:", loss.item())


BackWard Pass & Gradients

In [ ]:

model.zero_grad()

loss.backward()

print("Weight Gradients:\n", model.weight.grad)
print("Bias Gradients:\n", model.bias.grad)


PART 2

LOAD MNIST DATASET

In [ ]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False
)


NEURAL NETWORK MODEL

In [ ]:
class FeedForwardNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten image
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


LOSS FUNCTION & OPTIMIZER

In [ ]:
model = FeedForwardNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


TRAINING LOOP

In [ ]:
num_epochs = 3

for epoch in range(num_epochs):
    total_loss = 0

    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")


ACCURACY CALCULATION

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print("Test Accuracy:", accuracy, "%")
